This is based on the tensorflow tutorial. 

In [1]:
from __future__ import print_function,division
import random 
import tensorflow as tf
import numpy as np

C:\Users\chaor\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Tensorflow - Matrix Operations

All the following questions should be solved without using any of python's in-built control structures : for, while, if-else, etc. In addition to the tutorial material, you may find these functions useful:
* tf.cast
* tf.range
* tf.pow

Also, the following functions are used in later part: tf.sqrt and tf.square. 

### 1. Alternating addition

In [2]:
def alt_addition():
    """
        Given a 1d-matrix of 1 x n, compute the alternating addition.
        Ex: If A is
        1 2 3 4 5 6 7 8 9
        the return value should be 1 - 2 + 3 - 4 + 5 - 6 + 7 - 8 + 9.
        Your solution shouldn't include the use of tf.where or tf.boolean_mask. 
        It should include only mathematical operations.
    """
    A = tf.placeholder(dtype=tf.float64,shape=[1,None])
    # Write code to compute B.
    Bpos = A[:,::2]
    Bneg = tf.negative(A[:,1::2])
    B = tf.reduce_sum(Bpos) + tf.reduce_sum(Bneg)
    print(B)
    return A,B

In [3]:
A,B = alt_addition()

Tensor("add:0", shape=(), dtype=float64)


In [4]:
# Just run this cell after completing the function alt_addition.
def compute_alt(array):
    sum_ = 0
    for ind,i in enumerate(array):
        if ind%2 == 0:
            sum_ += i
        else:
            sum_ -= i
    return sum_
sess = tf.InteractiveSession()
for num in [5,10,13,20,100]:
    x = np.random.randn(num)
    b = sess.run(B,feed_dict={A:x.reshape([1,-1])})
    sum_ = compute_alt(x)
    if abs(b - sum_) < 10**-10:
        print("Correct!")
    else:
        print("Incorrect.")

Correct!
Correct!
Correct!
Correct!
Correct!


## 2. Odd Square and Even Cube

In [5]:
def sq_cube():
    """
    Given a 2-d matrix of dimensions N x N, square the odd elements and cube the even elements.
    Your solution shouldn't include the use of tf.where or tf.boolean_mask. 
    It should include only mathematical operations.
    """
    A = tf.placeholder(dtype=tf.float64,shape=[None,None])
    f = lambda x: (x*x*(1-(x%2))*x) + (x*x*(x%2)) #use mod of x to multiply/not-multiply by zero
    B = tf.map_fn(f, A)
    return A,B

In [6]:
A,B = sq_cube()

In [7]:
# Just run this cell after completing the function sq_cube.
def compute_sq_cube(array):
    m = [[0 for _ in range(len(array[0]))] for _ in range(len(array))]
    for row_ in range(len(array)):
        for col_ in range(len(array[row_])):
            m[row_][col_] = array[row_][col_]**3 if array[row_][col_]%2==0 else array[row_][col_]**2
    return m
sess = tf.InteractiveSession()
for num in [5,10,20]:
    x = np.array(range(num*num)).reshape([num,num])
    b = sess.run(B,feed_dict={A:x})
    res = compute_sq_cube(x.tolist())
    if (abs(b - res) < 10**-10).all():
        print("Correct!")
    else:
        print("Incorrect.")

Correct!
Correct!
Correct!


C:\Users\chaor\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


## 3. Tensorflow differentiation

In this question, you will learn about tf.gradients. Essentially, tf.gradients(p,q) gives dp/dq. 

In [8]:
x = tf.placeholder(tf.float32)
y = tf.sqrt(tf.square(x))
dy_dx = tf.gradients(y,x)

Essentially, y = x. And, as you know dy/dx = 1.  

In [9]:
sess = tf.Session()
sess.run(dy_dx,feed_dict={x:10})

[1.0]

And it is 1! Now, let's try for some other values

In [10]:
sess.run(dy_dx,feed_dict={x:0})

[nan]

In [11]:
sess.run(dy_dx,feed_dict={x:10**-20})

[inf]

Oops! We got nan and inf even though the derivative(limit) exists!! Explain why.

<b>Answer:</b> 
TensorFlow uses a differentiation method called "Automatic Differentiation". This method builds off of the idea that complex mathematical functions are composed of less complex math functions. For example, 

                  f = mx + b
                  
is composed of the simpler functions 

                  a = m*x 
                  &,
                  b = a + b

Thus, we can find the derivative of the original function f by finding the derivatives of the simpler functions and multiplying together (by using the chain rule). 

Where this idea goes wrong is when one of the "simpler" functions is not differentiable for the given input, as in the example of x = 0 in dy_dx. Even though the function 

                  f = sqrt(sq(x))

is the same as the absolute value of x, tensorflow does not see the function this way; it sees the function dy_dx as 

                   [(1/2)*(1/sqrt(x))]*[2*x]

Then, when 0 is inputted, 1/sqrt(0) does not exist, and hence we receive the value of NaN. There is a similar case for the returned [inf] value when inputting the value 10**-20.


References: 
-------------
* http://colah.github.io/posts/2015-08-Backprop/
* https://github.com/tensorflow/tensorflow/issues/5562



Further, come up with another function of x which shows similar behavior and demonstrate it below. Do not use trivial modifications of the above function like (x\*\*3)**1/3. In other words, demonstrate a function which gives the derivative as equal to nan/inf even though the derivative is defined. 

In [13]:
#Another function which displays the problems seen above. This is because the derivative of ln x is 1/x, and hence the derivative fails 
#in TF when x = 0. However, we can derive this function on paper as (x+2)/(x+1) 

u = tf.placeholder(tf.float32)
w = tf.log(tf.exp(x)+(x*tf.exp(x)))
dy_dx = tf.gradients(y,x)

In [14]:
sess = tf.Session()
sess.run(dy_dx,feed_dict={x:0})

[nan]

In [15]:
sess.run(dy_dx,feed_dict={x:10**-20})

[inf]

**The abs example**

In [0]:
x = tf.placeholder(tf.float32)
y = tf.abs(x)
dy_dx = tf.gradients(y,x)

In [4]:
sess = tf.Session()
sess.run(dy_dx,feed_dict={x:10})

[1.0]

In [5]:
sess.run(dy_dx,feed_dict={x:0})

[0.0]

In [6]:
sess.run(dy_dx,feed_dict={x:10**-20})

[1.0]

In [7]:
sess.run(dy_dx,feed_dict={x:-10})

[-1.0]